In [1]:
import pandas as pd
import geopandas as gpd
import numpy as np
from glob import glob
%matplotlib inline

In [3]:
sections = r"D:\Projects\Pr\2505.12\Eduvizig\Sections (waterkeringen)\geo_sections_4326_measure_join.shp"
station1 = r"D:\Projects\Pr\2505.12\Eduvizig\Sensor stations (waterstandlocaties)\stations_bak.shp"
station2 = r"D:\Projects\Pr\2505.12\Eduvizig\Sensor stations (waterstandlocaties)\stations.shp"
gdf_station1 = gpd.read_file(station1)
gdf_station2 = gpd.read_file(station2)
gdf_sections = gpd.read_file(sections)

In [4]:
station_join = pd.merge(gdf_station1, gdf_station2, left_on='name', right_on='name')
section_station_join = pd.merge(gdf_sections, station_join, left_on='stationid', right_on='id_y')[['stationid', 'id', 'min_waterl', 'max_waterl', 'threshold_', 'threshol_1', 'threshol_2']]

In [2]:
sectionids = [1, 2, 3, 4, 5, 6, 7, 8, 9,10,11,12,13]
stationids = [6, 5, 3, 2,12,13,11, 9,15, 7,17,9, 7]

station_section = pd.DataFrame(data={'stationid':stationids, 'sectionid':sectionids})
station_section

,stationid,sectionid
0,6,1
1,5,2
2,3,3
3,2,4
4,12,5
5,13,6
6,11,7
7,9,8
8,15,9
9,7,10


In [16]:
segmenteffects = []
for row in section_station_join.iterrows():
    loc = row[1]
    loc_info = {
        'stationid':loc.stationid,
        'sectionid':loc.id,
        'min':loc['min_waterl']/100, 
        'max':loc['max_waterl']/100, 
        '0_thd': loc['threshold_']/100, 
        '1_thd': loc['threshol_1']/100, 
        '2_thd': loc['threshol_2']/100
    }
    thd_level = {
        '0_thd':0.0005,#0/3, 
        '1_thd':0.005,#1/3, 
        '2_thd':0.05 #2/3
    }
    
    if loc_info['0_thd'] == 0 and loc_info['1_thd'] == 0 and loc_info['2_thd'] == 0:
        waterlevel_divid = (loc_info['max'] - loc_info['min']) / 8
        loc_info['0_thd'] = loc_info['min'] + (5 * waterlevel_divid)
        loc_info['1_thd'] = loc_info['min'] + (6 * waterlevel_divid)        
        loc_info['2_thd'] = loc_info['min'] + (7 * waterlevel_divid)                
    
    sel_station = station_section[station_section['sectionid']==loc_info['sectionid']]
    
    if sel_station['stationid'].iloc[0] == loc_info['stationid']:
        for k in ['max','2_thd','1_thd','0_thd']:
            segmenteffects.append({'sectionid':int(loc_info['sectionid']), 'segmentid':int(loc_info['sectionid']), 'upperboundary':loc_info[k]})

In [18]:
df_segmenteffects = pd.DataFrame(segmenteffects)#.sort_values()

In [19]:
df_segmenteffects.head()

,sectionid,segmentid,upperboundary
0,6,6,3.15
1,6,6,2.20
2,6,6,2.00
3,6,6,1.60
4,7,7,5.20


In [20]:
df_segmenteffects.sort_values(['sectionid','upperboundary'], inplace=True)

In [23]:
df_segmenteffects.head()

,sectionid,segmentid,upperboundary
43,1,1,5.00
42,1,1,6.00
41,1,1,6.50
40,1,1,8.13
35,2,2,5.00


In [22]:
df_segmenteffects.to_csv(r"P:\PR\3730.10\Werkzaamheden\PR3730.15_IT\Data\Eduvizig\segmenteffects_2.csv", index=False)